In [1]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

The 'env' object is used to:
1. Query for information about the environment
2. Sample states and actions
3. Retrieve rewards
4. Have the agent navigate the frozen lake

In [2]:
env = gym.make("FrozenLake-v1")

Construct Q-Table and initialize all Q values to 0 for each state action pair

In [3]:
# This will be the number of columns in the Q-Table
action_space_size = env.action_space.n

# This will be the number of rows in the Q-Table
state_space_size = env.observation_space.n

# Construct Q-Table with all zeros
q_table = np.zeros((state_space_size, action_space_size))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Initializate parameters needed to implement the Q-learning algorithm

- `num_episodes`: total number of episodes we want the agent to play during training
- `max_steps_per_episode`: max number of steps the agent is allowed to take within a single episode, after these many steps, the episode terminates

- `learning_rate`: alpha - how quickly the agent discards previous Q value for new Q value
- `discount_rate`: gamma - by what factor do we reduce the value of a future reward at time t+1

- `exploration_rate`: epsilon - probability that the agent will explore the environment
- `max_exploration_rate`: maximum epsilon
- `min_exploration_rate`: minimum epsilon
- `exploration_rate_decay`: the exporation rate will decay by this much in each episode

In [4]:
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.2
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

Implement the Q-learning algorithm

In [5]:
# List to hold the cumulative rewards from all episodes to see how the game scores change over time
rewards_all_episodes = []

# Q-learning algorithm
# For each episode
for episode in range(num_episodes):
    # Reset state to starting state
    state = env.reset()
    
    # This variable keeps track of whether or not the episode is finished
    done = False
    
    # Cumulative reward for this episode
    rewards_current_episode = 0
    
    # For each timestep in the episode
    for step in range(max_steps_per_episode):
        
        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        if(exploration_rate_threshold > exploration_rate):
            # Choose the action with the highest q value for this state (exploitation)
            action = np.argmax(q_table[state, :])
        else:
            # Choose a random action from the action space (exploration)
            action = env.action_space.sample()
        
        # Take the chosen action and receive the reward, next state, and other info
        new_state, reward, done, info = env.step(action)
        
        # Update Q-table for Q(s, a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        # Update state and cumulative reward
        state = new_state
        rewards_current_episode += reward
        
        # If the agent reached the goal or fell through a hole, end the episode
        if(done == True):
            break
        
    # Exploration rate decay for next episode with this formula (exponential decay)
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)
    
    # Append cumulative reward from this episode to the list of all cumulative rewards
    rewards_all_episodes.append(rewards_current_episode)
    
# Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
count = 1000
print("*******Average reward per thousand episodes*******\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

# Print updated Q-table
print("\n\n*******Q-table*******\n")
print(q_table)

*******Average reward per thousand episodes*******

1000 :  0.03800000000000003
2000 :  0.17900000000000013
3000 :  0.36400000000000027
4000 :  0.5410000000000004
5000 :  0.6350000000000005
6000 :  0.6580000000000005
7000 :  0.6730000000000005
8000 :  0.6410000000000005
9000 :  0.6860000000000005
10000 :  0.6740000000000005


*******Q-table*******

[[0.55574479 0.49143338 0.47938168 0.46480604]
 [0.26186033 0.23881211 0.31012792 0.49111991]
 [0.40119114 0.3850676  0.40894843 0.45908701]
 [0.34104128 0.34153173 0.38980491 0.44027588]
 [0.56565681 0.30418982 0.42132169 0.3605739 ]
 [0.         0.         0.         0.        ]
 [0.35577389 0.12354596 0.16055015 0.04681856]
 [0.         0.         0.         0.        ]
 [0.47083841 0.32177694 0.37793691 0.57907305]
 [0.46354762 0.62530725 0.41598707 0.42018161]
 [0.59337128 0.42083718 0.24956589 0.31677998]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.44838391 0.43252517 0.7016307  0.3

The average reward per 10000 episodes is improving. In the last 1000 episodes, the agent was winning ~67% of the time. Keep in mind that the ice is slipery and the agent doesn't always move in the direction it intends to move in. Therefore, the agent is learning.

#### Watching the Agent Play
The next block of code helps visualize how the trained agent plays the FrozenLake game.

In [6]:
# Watch it play 3 episodes
for episode in range(3):
    state = env.reset()
    
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)
    
    for step in range(max_steps_per_episode):
        # Clear output from this cell, wait until there is another output before clearing the current output
        clear_output(wait=True)
        
        # Render the current state of the environment to the display so that we can visually see it
        env.render()
        
        # Sleep for 0.3 seconds so we have time to see the observe state of the environment
        time.sleep(0.3)
        
        # Pick and take an action
        action = np.argmax(q_table[state,:])
        new_state, reward, done, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            
            # Render the environment to see where the agent ended up
            env.render()
            
            if(reward == 1):
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                print("****You fell through a hole!****")
                time.sleep(3)
            
            clear_output(wait=True)
            break
            
        state = new_state

env.close()

  (Down)
SFFF
FHFH
FFFH
HFFG
****You reached the goal!****
